In [54]:
import os
from dotenv import load_dotenv
from llama_index.core import (
    VectorStoreIndex, 
)
from llama_index.core.agent.workflow import AgentStream, ToolCallResult
load_dotenv()

True

In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.llms.google_genai import GoogleGenAI

llm_gemini = GoogleGenAI(model="gemini-2.0-flash", api_key=os.environ["GEMINI_API_KEY"])

In [57]:
import pandas as pd

# dataset con diferentes clientes, su email y el detalle de una compra
df_shoppings = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "DetalleCompra": [
        "Compra de laptop - Modelo: Dell XPS 13, Procesador: Intel i7, RAM: 16GB, Almacenamiento: 512GB SSD",
        "Compra de smartphone - Modelo: iPhone 14 Pro, Almacenamiento: 256GB, Color: Negro",
        "Compra de tablet - Modelo: Samsung Galaxy Tab S8, Pantalla: 11 pulgadas, Almacenamiento: 128GB",
        "Compra de audífonos - Modelo: Sony WH-1000XM5, Tipo: Over-ear, Cancelación de ruido activa",
        "Compra de monitor - Modelo: LG UltraGear 27GP850, Tamaño: 27 pulgadas, Resolución: QHD, Frecuencia: 165Hz"
    ]
})

# dataset con diferentes clientes, su email y el detalle de un ticket
df_tickets = pd.DataFrame({
    "Cliente": ["Juan Pérez", "Ana Gómez", "Luis Martínez", "Ana Gómez", "Carlos López"],
    "Email": ["juan.perez@email.com", "ana.gomez@email.com", "luis.martinez@email.com", "ana.gomez@email.com", "carlos.lopez@email.com"],
    "idTicket": [1, 2, 3, 4, 5],
    "DetalleTickets": [
        "Problema con la batería del laptop, no carga correctamente. Modelo: Dell XPS 13, Fecha de compra: 2023-01-15, Estado: En garantía.",
        "Pantalla del smartphone presenta líneas verticales. Modelo: iPhone 14 Pro, Fecha de compra: 2023-02-10, Estado: En garantía.",
        "Tablet no enciende después de la actualización. Modelo: Samsung Galaxy Tab S8, Fecha de compra: 2023-03-05, Estado: Fuera de garantía.",
        "Audífonos no se conectan vía Bluetooth. Modelo: Sony WH-1000XM5, Fecha de compra: 2023-04-20, Estado: En garantía.",
        "Monitor tiene píxeles muertos en la esquina superior derecha. Modelo: LG UltraGear 27GP850, Fecha de compra: 2023-05-30, Estado: En garantía."
    ]
})

In [45]:
from llama_index.core.schema import Document

documents_shoppings = [
    Document(
        text=f"Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleCompra']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"]},
    )
    for record in df_shoppings.to_dict(orient="records")
]

documents_tickets = [
    Document(
        text=f"Ticket ID: {record['idTicket']}, Cliente: {record['Cliente']}, Email: {record['Email']}, Detalle: {record['DetalleTickets']}",
        metadata={"Cliente": record["Cliente"], "Email": record["Email"], "idTicket": record["idTicket"]},
    )
    for record in df_tickets.to_dict(orient="records")
]

index_shoppings = VectorStoreIndex.from_documents(documents_shoppings, show_progress=True)
index_tickets = VectorStoreIndex.from_documents(documents_tickets, show_progress=True)

Generating embeddings: 100%|██████████| 5/5 [00:00<00:00, 23.79it/s]


In [46]:
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters
from google import genai
from google.genai import types
from pydantic import BaseModel, Field

class SentimentSchema(BaseModel):
    """Sentiment analysis schema"""

    sentimiento: str = Field(..., description="Text sentiment")
    score: str = Field(
        ..., description="Sentiment score (0-1). Higher score means a more positive sentiment"
    )

def analyze_sentiment(text: str) -> dict:
    """Analyze the sentiment (positive, negative, neutral) of a given text."""

    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=text,
        config=types.GenerateContentConfig(
            system_instruction="""You are a sentiment analysis expert. Your task is to analyze the sentiment of the text provided.
            You must follow `SentimentSchema` pydantic model to return the response.""",
            response_mime_type="application/json",
            response_schema=SentimentSchema,
        ),
    )
    return response.text

sentiment_tool = FunctionTool.from_defaults(
    fn=analyze_sentiment,
    name="SentimentAnalyzer",
    description="Analiza emociones en textos de clientes"
)

def search_ticket_by_id(id_ticket: int) -> str:
    """Busca un ticket específico por su ID exacto
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    id_ticket : int
        ID del ticket a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        id_ticket = int(id_ticket)
        filters = MetadataFilters(filters=[MetadataFilter(key="idTicket", value=id_ticket)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket con este ID exacto."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket con ID {id_ticket}. Error: {str(e)}"

search_ticket_by_id_tool = FunctionTool.from_defaults(
    fn=search_ticket_by_id,
    name="search_ticket_by_id",
    description="Busca información específica de un ticket usando su número ID exacto"
)

def search_ticket_by_email(email_client: str) -> str:
    """Busca un ticket específico por el email del cliente
    y devuelve un resumen detallado del problema reportado.

    Parameters
    ----------
    email_client : str
        Email del cliente a buscar.

    Returns
    -------
    str
        Resumen detallado del ticket.
    """
    try:
        filters = MetadataFilters(filters=[MetadataFilter(key="Email", value=email_client)])
        engine = index_tickets.as_query_engine(
            filters=filters,
            response_mode="tree_summarize",
            llm=llm_gemini,
            system_prompt="Encuentra y resume el contenido del ticket correspondiente al cliente con este email."
        )
        return engine.query("Describe detalladamente este ticket").response
    except Exception as e:
        return f"No se encontró el ticket del cliente con email {email_client}. Error: {str(e)}"

search_ticket_by_email_tool = FunctionTool.from_defaults(
    fn=search_ticket_by_email,
    name="search_ticket_by_email",
    description="Busca información específica de un ticket usando el email del cliente al que pertenece el ticket"
)

In [47]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

duckduckgo_search_tool = [
    tool
    for tool in DuckDuckGoSearchToolSpec().to_tool_list()
    if tool.metadata.name == "duckduckgo_full_search"
]

In [48]:
response_mode = "refine"

query_engine_shoppings = index_shoppings.as_query_engine(
    response_mode=response_mode,
    verbose=False, 
    system_prompt="Eres un asistente de atención al cliente. Responde preguntas sobre el historial de compras de clientes.", 
    similarity_top_k=2, 
    llm=llm_gemini
)

crm_shoppings_rag_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_shoppings,
    name="crm_shoppings_rag_tool",
    description="""Herramienta de búsqueda para responder preguntas sobre el historial de compras de clientes. 
    Úsalo cuando necesites información sobre productos comprados anteriormente""",
)

query_engine_tickets = index_tickets.as_query_engine(
    response_mode=response_mode,
    verbose=False, 
    system_prompt="Eres un asistente de atención al cliente. Responde preguntas sobre el historial de tickets clientes.", 
    similarity_top_k=2, 
    llm=llm_gemini,
)

crm_tickets_rag_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_tickets,
    name="crm_tickets_rag_tool",
    description="""Herramienta de búsqueda para responder preguntas sobre el historial de tickets clientes.
    Úsalo cuando necesites información sobre problemas reportados por clientes""",
)

In [49]:
tools = [sentiment_tool, crm_shoppings_rag_tool, crm_tickets_rag_tool, search_ticket_by_id_tool, search_ticket_by_email_tool, duckduckgo_search_tool[0]]

from ReAct_system_prompt import react_system_prompt
from llama_index.core import PromptTemplate

special_instructions = """
Your duty as an agent is to help customers solve problems related to their purchases and tickets.
You must answer customers' questions using the available ticket and purchase information.

First of all, you should analyze the sentiment of the customers' messages.
If the sentiment is negative, you should look for information about the customer's ticket.

If there is a ticket, you should make a web search, through the `duckduckgo_search_tool`, to find and compare, first of all the market price of the product, and secondly the cost of the repair.
IMPORTAR: Always use valid JSON format with double quotes for keys and values.

Finally, based on the customer's question, you should write a formal email-type response, including relevant ticket and purchase information
and a possible solution to the problem raised.

In case you need for information to answer the question, you can ask the user for more details.
"""

# Create the agent with the tools
agent = ReActAgent(
    tools=tools,
    llm=llm_gemini,
    # system_prompt=[...], # Doesn't behave as expected
)

# Actualiza el prompt del sistema
agent.update_prompts({"react_header": PromptTemplate(react_system_prompt.replace("$/special_instructions/$", special_instructions))})

ctx = Context(agent)

In [ ]:
query = "Me ha llegado el portatil roto. luis.martinez@email.com"

handler = agent.run(query, ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

In [53]:
print(f"\n\nFinal response: \n{str(response)}")



Final response: 
Lo siento, no puedo acceder a la información de su compra o ticket con las herramientas disponibles. Por favor, proporcione el número de orden de compra o el número de ticket para que pueda ayudarle mejor.


In [ ]:
print(f"\n\nFinal response: \n{response}")

In [ ]:
handler = agent.run("Me gustaría repararla, cuanto me costaría?", ctx=ctx)
response = await handler
print(response)

El costo de reparación de un Samsung Galaxy Tab S8 depende del daño específico. Por ejemplo, el reemplazo de la batería podría costar alrededor de 90 euros. Si el problema es con la pantalla, el costo podría ser mayor, especialmente si el panel AMOLED está dañado. Para obtener una estimación precisa, te recomiendo contactar un servicio técnico autorizado de Samsung o una tienda de reparación especializada.

Aún necesito que me confirmes si el "portátil roto" es el mismo Samsung Galaxy Tab S8 que aparece en el ticket ID 3. Si es un dispositivo diferente, por favor, proporciona más detalles sobre el modelo del portátil para poder buscar información más precisa sobre su reparación.


In [ ]:
handler = agent.run("Quiero proceder con la reparación", ctx=ctx)
response = await handler
print(response)

Entiendo que quieres proceder con la reparación. Sin embargo, antes de continuar, necesito confirmar si el "portátil roto" es el mismo dispositivo (Samsung Galaxy Tab S8) que mencionaste en el ticket ID 3. ¿Podrías confirmarme si se trata del mismo dispositivo o de uno diferente? Si es un dispositivo diferente, por favor, proporciona la marca y modelo del portátil para poder ayudarte mejor con el proceso de reparación.
